In [ ]:
!python -m pip install git+https://github.com/huggingface/transformers
! pip install openpyxl

  Cloning https://github.com/huggingface/transformers to /tmp/pip-req-build-71xicip6
  Running command git clone --filter=blob:none --quiet https://github.com/huggingface/transformers /tmp/pip-req-build-71xicip6
  Resolved https://github.com/huggingface/transformers to commit 816f4424964c1a1631e303b663fc3d68f731e923
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.0/3.0 MB 31.7 MB/s eta 0:00:00
  Created wheel for transformers: filename=transformers-4.46.0.dev0-py3-none-any.whl size=9991549 sha256=43780e19cd705d729aa306f49453f239099af5b620abd073486a058265f0269f
  Stored in directory: /tmp/pip-ephem-wheel-cache-lof3gacq/wheels/c0/14/d6/6c9a5582d2ac191ec0a483be151a4495fe1eb2a6706ca49f1b
Successfully built transformers
  Attempting uninstall: tokenizers
    Found existing installation: tokenizers 0.19.1
    Uninstalling tokenizers-0.19.1:
      Successfully u

In [ ]:
import re
import openpyxl
import cv2
from PIL import Image
import torch
from transformers import Qwen2VLForConditionalGeneration, AutoTokenizer, AutoProcessor
import numpy as np
from google.colab import files
import os

In [ ]:
# Load the Qwen2 model and processor
model = Qwen2VLForConditionalGeneration.from_pretrained(
    "Qwen/Qwen2-VL-2B-Instruct",
    torch_dtype="auto",
    device_map="auto",
)

processor = AutoProcessor.from_pretrained("Qwen/Qwen2-VL-2B-Instruct")

# Define the Excel file path
file_path = "product_analysis.xlsx"

if os.path.exists(file_path):
    # Open the existing workbook
    workbook = openpyxl.load_workbook(file_path)
    sheet = workbook.active
else:
    # Create a new workbook
    workbook = openpyxl.Workbook()
    sheet = workbook.active
    sheet.title = "Product Analysis"

    # Add headers
    headers = ["Product Name", "Category", "Quantity", "Count", "Expiry Date", "Freshness Index", "Shelf Life"]
    sheet.append(headers)

# Regular expression patterns to extract data
packaged_product_pattern = r"Product Name: (.*)\n  - Product Category: (.*)\n  - Product Quantity: (.*)\n  - Product Count: (.*)\n  - Expiry Date: (.*)"
fruits_vegetables_pattern = r"Type of fruit/vegetable: (.*)\n  - Freshness Index: (.*)\n  - Estimated Shelf Life: (.*)"


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

In [ ]:
# Let user upload the image
uploaded = files.upload()

In [ ]:
# Iterate over all uploaded files
for image_name in uploaded.keys():
    # Open the image using PIL
    image = Image.open(image_name)
    image = image.resize((256, 256))

    # Prepare the text prompt for predicting product details and freshness
    messages = [
        {
            "role": "user",
            "content": [
                {
                    "type": "image",
                    "image_url": "Captured from webcam"  # Description for internal use
                },
                {
                    "type": "text",
                    "text": """This image contains fruits, vegetables, or packaged products.
                    Please analyze the image and provide:
                    - For products or fruits or vegetables:
                        - Product Name
                        - Product Category
                        - Product Quantity
                        - Product Count
                        - Expiry Date (if available)
                        - Freshness Index (based on visual cues)
                        - Estimated Shelf Life"""
                }
            ]
        }
    ]

    # Prepare the text prompt for processing
    text_prompt = processor.apply_chat_template(messages, add_generation_prompt=True)

    # Process the image and prompt for model input
    inputs = processor(
        text=[text_prompt],
        images=[image],
        padding=True,
        return_tensors="pt"
    )

    # Move inputs to GPU if available
    inputs = inputs.to("cuda" if torch.cuda.is_available() else "cpu")

    # Generate output from the model
    output_ids = model.generate(**inputs, max_new_tokens=1024)

    # Decode the generated output
    generated_ids = [
        output_ids[len(input_ids):]
        for input_ids, output_ids in zip(inputs.input_ids, output_ids)
    ]
    output_text = processor.batch_decode(
        generated_ids, skip_special_tokens=True, clean_up_tokenization_spaces=True
    )[0]

    # Print the output text for the current image
    print(f"Analysis Output for the captured frame:")
    print(output_text)

    # Improved regex pattern for extracting product information
    product_pattern = re.compile(
        r"Product Name\s*:\s*(.*?)\s*"
        r"Product Category\s*:\s*(.*?)\s*"
        r"Product Quantity\s*:\s*(.*?)\s*"
        r"Product Count\s*:\s*(.*?)\s*"
        r"Expiry Date\s*:\s*(.*?)\s*"
        r"Freshness Index\s*:\s*(.*?)\s*"
        r"Shelf Life\s*:\s*(.*?)\s*",
        re.IGNORECASE | re.DOTALL
    )

    # Extract packaged product information using regex
    product_match = re.search(product_pattern, output_text)

    if product_match:
        product_name = product_match.group(1).strip()
        category = product_match.group(2).strip()
        quantity = product_match.group(3).strip()
        count = product_match.group(4).strip()
        expiry_date = product_match.group(5).strip()
        freshness_index = product_match.group(6).strip()
        shelf_life = product_match.group(7).strip()
    else:
        # If no match is found, default to "N/A"
        product_name = category = quantity = count = expiry_date = freshness_index = shelf_life = "N/A"

    # Insert row into Excel
    sheet.append([product_name, category, quantity, count, expiry_date, freshness_index, shelf_life])

# Save the workbook
workbook.save("product_analysis.xlsx")
print("Data saved to product_analysis.xlsx")


Analysis Output for the captured frame:
- Product Name: Panga Masala
- Product Category: Spices
- Product Quantity: 1 pack
- Product Count: 1
- Expiry Date: Not visible in the image
- Freshness Index: Not applicable
- Estimated Shelf Life: Not applicable
Analysis Output for the captured frame:
- Product Name: Nestlé Choco Chips
- Product Category: Snacks
- Product Quantity: 1 pack
- Product Count: 1
- Expiry Date: Not visible in the image
- Freshness Index: Based on visual cues, the product appears to be fresh and intact, indicating a high freshness index.
- Estimated Shelf Life: The product is likely to remain fresh for several months after opening, depending on storage conditions.
Analysis Output for the captured frame:
- Product Name: Oatmeal
- Product Category: Breakfast Cereal
- Product Quantity: 100g
- Product Count: 1
- Expiry Date: Not visible in the image
- Freshness Index: Based on visual cues, the product appears to be fresh and well-preserved.
- Estimated Shelf Life: 2-3 da